<a href="https://colab.research.google.com/github/ZeroOFBright/Spore-Model/blob/main/Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# จัดการเรื่องที่อยู่ไฟล์



In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd /content/drive/My\ Drive/bot
!pwd

/content/drive/My Drive/bot
/content/drive/My Drive/bot


In [3]:
!ls

DIgits_seg_mrcnn	      maskrcnn_config20200917T1219
input			      maskrcnn_config20210204T0821
Mask_RCNN		      maskrcnn_config20210204T0826
mask_rcnn_01.h5		      maskrcnn_config20210204T0833
maskrcnn_config20200501T1031  maskrcnn_config20210204T0840
maskrcnn_config20200501T1048  maskrcnn_config20210304T1636
maskrcnn_config20200907T1847  mrcnn
maskrcnn_config20200908T1703


In [4]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

!pip install q keras==2.1.6

TensorFlow 1.x selected.
1.15.2


In [5]:
from tensorflow.python.client import device_lib
import tensorflow as tf
tf.test.gpu_device_name()
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15719899829793746018, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8528908906125389627
 physical_device_desc: "device: XLA_CPU device"]

# import model

In [6]:
# import library
import mrcnn
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
from mrcnn.utils import Dataset
from mrcnn.model import MaskRCNN

import numpy as np
from numpy import zeros
from numpy import asarray
import colorsys
import argparse
import imutils
import random
import cv2
import os
import time
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from keras.models import load_model
from os import listdir
from xml.etree import ElementTree
%matplotlib inline
#ตั้งค่าให้ code ใช้ CUDA_VISIBLE_DEVICES ของ GPU ในการ Train ข้อมูล
os.environ["CUDA_DEVICE_ORDER"]="0000:01:00.0"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


Using TensorFlow backend.


In [7]:
class myMaskRCNNConfig(Config):
    NAME = "MaskRCNN_config"
 
    # set the number of GPUs to use along with the number of images
    # per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # number of classes (we would normally add +1 for the background)
     # kangaroo + BG
    NUM_CLASSES = 1+1
   
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 131
    
    # Learning rate
    LEARNING_RATE=0.006
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=10

config = myMaskRCNNConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.006
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [8]:
class monster_Dataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        # Add classes. We have only one class to add.
        self.add_class("dataset", 1, "spores")

        # define data locations for images and annotations
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # Iterate through all files in the folder to 
        #add class, images and annotaions
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4]
            # setting image file
            img_path = images_dir + filename
            
            # setting annotations file
            ann_path = annotations_dir + image_id + '.xml'
            # adding images and annotations to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
            
  # extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        
        # define anntation  file location
        path = info['annotation']
        
        # load XML
        boxes, w, h = self.extract_boxes(path)
       
        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('spores'))
        return masks, asarray(class_ids, dtype='int32')

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        print(info)
        return info['path']

# เตรียม datasetสำหรับtrain

In [9]:
# prepare train set
train_set = monster_Dataset()
train_set.load_dataset('input/train' )
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
valid_set = monster_Dataset()
valid_set.load_dataset('input/validation')
valid_set.prepare()
print('Valid: %d' % len(valid_set.image_ids))
# prepare test/val set
sample_test_set = monster_Dataset()
sample_test_set.load_dataset('input/sample_test')
sample_test_set.prepare()
print('Test: %d' % len(sample_test_set.image_ids))

Train: 12
Valid: 4
Test: 4


In [10]:
model_dir='./'

In [11]:
print("Loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')

Loading Mask R-CNN model...





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [12]:
os.listdir("./")

['mrcnn',
 'Mask_RCNN',
 'maskrcnn_config20200501T1031',
 'maskrcnn_config20200501T1048',
 'input',
 'maskrcnn_config20200907T1847',
 'maskrcnn_config20200908T1703',
 'maskrcnn_config20200917T1219',
 'maskrcnn_config20210204T0821',
 'maskrcnn_config20210204T0826',
 'maskrcnn_config20210204T0833',
 'maskrcnn_config20210204T0840',
 'mask_rcnn_01.h5',
 'maskrcnn_config20210304T1636',
 'DIgits_seg_mrcnn']

In [13]:
from tensorflow.python.keras.saving import hdf5_format

In [14]:
#load the weights for COCO
model.load_weights('Mask_RCNN/mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
## train heads with higher lr to speedup the learning
#ทำtransferlearning โดยเลือกให้trainเฉพาะlayersท้ายๆ
model.train(train_set, valid_set,learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')
history = model.keras_model.history.history


Starting at epoch 0. LR=0.006

Checkpoint Path: ./maskrcnn_config20210304T1641/mask_rcnn_maskrcnn_config_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4   

/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/5
 11/131 [=>............................] - ETA: 1:46:15 - loss: 1.9939 - rpn_class_loss: 0.0381 - rpn_bbox_loss: 0.3021 - mrcnn_class_loss: 0.2449 - mrcnn_bbox_loss: 0.9357 - mrcnn_mask_loss: 0.4731

In [ ]:
# Save weight ที่เราtrainมาแล้ว
model_path = 'mask_rcnn_01.h5'
model.keras_model.save_weights('mask_rcnn_01.h5')

In [ ]:
!ls

# นำmodel ที่trainมาทดสอบ

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

#โหลด โมเดลที่เราพึ่งเทรนไปล่าสุด
model_path = 'mask_rcnn_01.h5'
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=model_dir)

# loading the trained weights o the custom dataset
model.load_weights(model_path, by_name=True)

In [ ]:
#เลือกภาพที่ 2 จากvalidation set เอามาทำนาย mask และ box ดู
image_id = 1
image, image_meta, gt_class_id, gt_bbox, gt_mask =    modellib.load_image_gt(valid_set, config, image_id, use_mini_mask=False)
info = valid_set.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       valid_set.image_reference(image_id)))
# Run object detection
results = model.detect([image], verbose=1)

# Display results

r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            valid_set.class_names, r['scores'],
                            title="Predictions")

# ทดสอบเทสด้วยรูปที่เก็บไว้ใน sample test

In [ ]:
image_id = 1
image, image_meta, gt_class_id, gt_bbox, gt_mask =    modellib.load_image_gt(sample_test_set, config, image_id, use_mini_mask=False)
info = sample_test_set.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       sample_test_set.image_reference(image_id)))
# Run object detection
results = model.detect([image], verbose=1)

# Display results

r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            sample_test_set.class_names, r['scores'],
                            title="Predictions")